### Get reactions, extern reactions and metabolites from mmsyn models

In [1]:
import cobra
import numpy as np
import pandas as pd
import json
import os

#### Functions

In [2]:
def indicate_exchange_silent(model, extern_compartments):
    """Iterates over each reaction in the cobra model and adds an reaction.exchange attribute (value: True or False) to each reaction.
    This attribute is used later in the code to identify exchange reactions."""
    # if no SBO term and products and reactands on reaction --> cobrapy cant detect it as extern
    # we take each reaction as external if:
    # SBO:0000627
    # OR
    # reaction id starts with 'R_EX_' or 'EX_'
    # OR
    # reaction is one sided (reaction.boundary = True) and no demand or sink (we proof for demand and sink before we proof for boundary)
    # if exchange reaction, than reaction.exchange = True
    for reaction in model.reactions:
        # reaction id starts with 'R_EX_' or 'EX_'
        if 'EX_' in reaction.id[:5]:
            #print(f'exchange: {reaction.id}')
            reaction.exchange = True
            continue

        # unfortunately, demand reactions do have the same SBO Term as exchange reactions
        # demand reactions differ from exchange reactions in case of exchange reactions work with metabolites of external compartment
        # we could use that (if exchange sbo term --> proof if one metabolite is in external compartment):
        if 'sbo' in reaction.annotation:
            if reaction.annotation['sbo'] == 'SBO:0000627':
                extern = False
                for metabolite in reaction.metabolites:
                    # compartment name needs to be given and contain 'ex' or compartment id needs to contain 'e'
                    if (metabolite.compartment in extern_compartments and model.compartments[metabolite.compartment]) or 'e' in metabolite.compartment:
                        extern = True
                if extern:
                    # add reaction to exchange reactions
                    #print(f'exchange: {reaction.id}')
                    reaction.exchange = True
                    continue


        # kill all demand reactions by id prefix 'R_DM_'
        if 'DM_' in reaction.id[:5]:
            #print(f'demand: {reaction.id}')
            reaction.exchange = False
            continue
        # kill all sink reactions by SBO Term
        if 'sbo' in reaction.annotation:
            if reaction.annotation['sbo'] == 'SBO:0000632':
                #print(f'sink: {reaction.id}')
                reaction.exchange = False
                continue
        # kill all sink reactions by id prefix 'R_SINK_'
        if 'SINK_' in reaction.id[:6]:
            #print(f'sink: {reaction.id}')
            reaction.exchange = False
            continue

        # proof if one sided
        if reaction.boundary:
            # since we already ended iteration if criterium was for sink or demand, all remaining reaction.boundary = True reactions hopefully are exchange reactions
            #print(f'exchange: {reaction.id}')
            reaction.exchange = True
            continue
        # if no exchange reaction (iteration reaches this point)
        reaction.exchange = False

In [3]:
def indicate_exchange(model, extern_compartments):
    """Iterates over each reaction in the cobra model and adds an reaction.exchange attribute (value: True or False) to each reaction.
    This attribute is used later in the code to identify exchange reactions.
    :param model: cobrapy model
    :param list extern_compartments: list of extern compartments created by function get_extern_compartments
    :return: None
    """
    # if no SBO term and products and reactands on reaction --> cobrapy cant detect it as extern
    # we take each reaction as external if:
    # SBO:0000627
    # OR
    # reaction id starts with 'R_EX_' or 'EX_'
    # OR
    # reaction is one sided (reaction.boundary = True) and no demand or sink (we proof for demand and sink before we proof for boundary)
    # if exchange reaction, than reaction.exchange = True
    for reaction in model.reactions:
        # reaction id starts with 'R_EX_' or 'EX_'
        if 'EX_' in reaction.id[:5]:
            print(f'exchange: {reaction.id}')
            reaction.exchange = True
            continue

        # unfortunately, demand reactions do have the same SBO Term as exchange reactions
        # demand reactions differ from exchange reactions in case of exchange reactions work with metabolites of external compartment
        # we could use that (if exchange sbo term --> proof if one metabolite is in external compartment):
        if 'sbo' in reaction.annotation:
            if reaction.annotation['sbo'] == 'SBO:0000627':
                extern = False
                for metabolite in reaction.metabolites:
                    # compartment name needs to be given and contain 'ex' or compartment id needs to contain 'e'
                    if (metabolite.compartment in extern_compartments and model.compartments[metabolite.compartment]) or 'e' in metabolite.compartment:
                        extern = True
                if extern:
                    # add reaction to exchange reactions
                    print(f'exchange: {reaction.id}')
                    reaction.exchange = True
                    continue


        # kill all demand reactions by id prefix 'R_DM_'
        if 'DM_' in reaction.id[:5]:
            print(f'demand: {reaction.id}')
            reaction.exchange = False
            continue
        # kill all sink reactions by SBO Term
        if 'sbo' in reaction.annotation:
            if reaction.annotation['sbo'] == 'SBO:0000632':
                print(f'sink: {reaction.id}')
                reaction.exchange = False
                continue
        # kill all sink reactions by id prefix 'R_SINK_'
        if 'SINK_' in reaction.id[:6]:
            print(f'sink: {reaction.id}')
            reaction.exchange = False
            continue

        # proof if one sided
        if reaction.boundary:
            # since we already ended iteration if criterium was for sink or demand, all remaining reaction.boundary = True reactions hopefully are exchange reactions
            print(f'exchange: {reaction.id}')
            reaction.exchange = True
            continue
        # if no exchange reaction (iteration reaches this point)
        reaction.exchange = False

In [4]:
def get_extern_compartments(model):
    '''Writes all compartments in list which have "ex" in their name or no name to get all extern compartments. Takes also all compartments with no name, because metabolites get filtered after that for id in oter function.'''
    # we list all compartment ids of the model which have "ex" in their name or no name
    return [compartment for compartment in model.compartments if "ex" in model.compartments[compartment].lower() or not model.compartments[compartment]]

In [5]:
def count_ex_metabolites(model, extern_compartments):
    """Creates a list with all extern metabolites and retunrs count of extern metabolites."""
    # want to get all external metabolites
    # mplrs kicked out all internal ones already but our header does not know which
    # so we need to kick again
    new_ex_metabolites = []
    for reaction in model.reactions:
        if reaction.exchange:
            if reaction.reactants and reaction.products:
                # if exchange reactions are both sided in the model, we search only for metabolites with extern compartment
                # id of extern compartment needs to contain an "e" and compartment name needs "ex" or nothing, otherwise it will not work
                new_ex_metabolites += [metabolite.id for metabolite in reaction.metabolites.keys() if "e" in metabolite.compartment and metabolite.compartment in extern_compartments]
            else:
                new_ex_metabolites += [metabolite.id for metabolite in reaction.metabolites.keys()]

    # in case some external reactions share the same external metabolite and bring it in right order
    # right order is not necessary for ex_metabolites
    new_ex_metabolites = set(new_ex_metabolites)
    return len(new_ex_metabolites)

#### Get data table for all models

In [6]:
mmsyn_models = ['mmsyn_sm' + str(i).zfill(2) + '.xml' for i in range(0,23)]
models = ['e_coli_core.xml'] + mmsyn_models
root = './'
path = root + '/../metabolic_models/'

In [7]:
models

['e_coli_core.xml',
 'mmsyn_sm00.xml',
 'mmsyn_sm01.xml',
 'mmsyn_sm02.xml',
 'mmsyn_sm03.xml',
 'mmsyn_sm04.xml',
 'mmsyn_sm05.xml',
 'mmsyn_sm06.xml',
 'mmsyn_sm07.xml',
 'mmsyn_sm08.xml',
 'mmsyn_sm09.xml',
 'mmsyn_sm10.xml',
 'mmsyn_sm11.xml',
 'mmsyn_sm12.xml',
 'mmsyn_sm13.xml',
 'mmsyn_sm14.xml',
 'mmsyn_sm15.xml',
 'mmsyn_sm16.xml',
 'mmsyn_sm17.xml',
 'mmsyn_sm18.xml',
 'mmsyn_sm19.xml',
 'mmsyn_sm20.xml',
 'mmsyn_sm21.xml',
 'mmsyn_sm22.xml']

In [8]:
matrix = np.empty((len(models), 4), dtype=int)
for model, i in zip(models, range(len(models) + 1)):
    cobra_model = cobra.io.sbml.read_sbml_model(path + model, number=float)
    extern_compartments = get_extern_compartments(cobra_model)
    indicate_exchange_silent(cobra_model, extern_compartments)
    
    num_ex_metabolites = count_ex_metabolites(cobra_model, extern_compartments)
    reactions = len(cobra_model.reactions)
    metabolites = len(cobra_model.metabolites)
    extern_reactions = 0
    for reaction in cobra_model.reactions:
        if reaction.exchange:
            extern_reactions += 1
    
    vector = [reactions, extern_reactions, metabolites, num_ex_metabolites]
    matrix[i] = vector

In [9]:
table = pd.DataFrame(data=matrix, index=models, columns=['reactions', 'extern reactions', 'metabolites', 'extern metabolites'], dtype=int)

In [10]:
table.to_csv(path_or_buf='./n_reactions_metabolites.csv', sep=',')

In [11]:
# add data to complexity level
complexity_list = [str(int(model.split('.')[0].split('sm')[1])) if ('mmsyn_sm' in model) else 'NaN' for model in models]

In [12]:
# add 'complexity level' column to dataframe
table.insert(loc = 0, column = 'complexity level', value = complexity_list)
# add 'ECMs' column to dataframe
table['ECMs'] = 0

In [13]:
# implement ECM counts of result files
path = root + '/times/ECMproject/process/'
for model in models:
    model_name = model.split('.')[0]
    filename = path + model_name + '_mplrs_project_result_times.json'
    dictionary = json.load(open(filename))
    # get all ECM counts for one model
    ECM_counts = [dictionary[key]['ECM_count']['all_values'] for key in dictionary.keys()]
    ECM_count = []
    for count in ECM_counts:
        # proof if all values are the same
        count = set(count)
        if len(count) > 1:
            raise Exception(f'ECM count is not always the same in {model} in same number of processes!')
        count = list(count)[0]
        ECM_count.append(count)
    # proof if there is more than one different entry in list
    ECM_count = set(ECM_count)
    if len(ECM_count) > 1:
        raise Exception('ECM count is not always the same in {model} accross different process numbers!')
    ECM_count = list(ECM_count)[0]
    # write ECM count to pandas dataframe
    table['ECMs'][model] = ECM_count

/tmp/ipykernel_3339038/3729091461.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['ECMs'][model] = ECM_count


In [14]:
# change in latex format
print(table.to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} & complexity level &  reactions &  extern reactions &  metabolites &  extern metabolites &        ECMs \\
\midrule
e\_coli\_core.xml &              NaN &         95 &                20 &           72 &                  20 &         689 \\
mmsyn\_sm00.xml  &                0 &        253 &                56 &          245 &                  56 &        4132 \\
mmsyn\_sm01.xml  &                1 &        255 &                57 &          246 &                  57 &        6486 \\
mmsyn\_sm02.xml  &                2 &        257 &                58 &          247 &                  58 &        9702 \\
mmsyn\_sm03.xml  &                3 &        260 &                59 &          249 &                  59 &       19394 \\
mmsyn\_sm04.xml  &                4 &        263 &                60 &          251 &                  60 &       38778 \\
mmsyn\_sm05.xml  &                5 &        266 &                61 &          253 &                  61 &  

/tmp/ipykernel_3339038/1240624988.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table.to_latex())


### Get result table of all result files

#### ECMproject in default

In [15]:
root = './'
path = root + 'times/ECMproject/process/'
mmsyn_models = ['mmsyn_sm' + str(i).zfill(2) + '.xml' for i in range(0,23)]
models = ['e_coli_core.xml'] + mmsyn_models

In [16]:
dataframe = pd.DataFrame()
model_names = []

for model in models:
    model_name = model.split('.')[0]
    filename = path + model_name + '_own.csv'
    table = pd.read_csv(filename)
    for index in table.index:
        model_names.append(model_name)
    dataframe = pd.concat([dataframe, table])

# set new multiindex
dataframe['models'] = model_names
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index(['models', 'cores'])
dataframe

index     time_total  time_initial_setup  time_projection  \
models      cores                                                              
e_coli_core 3          0      87.360809            1.698161        80.968351   
            5          1     101.300994            1.501474        97.526205   
            10         2     117.173548            1.508881       113.598551   
            20         3     100.058802            1.507272        96.795638   
            30         4      98.629254            1.632936        94.993040   
            40         5     124.875052            1.554184       121.589994   
            60         6     102.363152            1.621820        98.930396   
mmsyn_sm00  3          0     381.487886           17.788558       354.835988   
            5          1     279.941254            8.448166       264.988741   
            10         2     246.097487            5.563633       234.413655   
            20         3     253.545279            5.595577       241.865454   
            30         4     262.162710            5.692996       250.411425   
            40         5     268.893921            5.732902       257.064799   
            50         6     285.816724            4.631251       275.070867   
            60         7     284.306541            4.736041       273.406994   
mmsyn_sm01  20         0     256.252435            5.640659       244.329035   
mmsyn_sm02  20         0     292.340747            5.888542       279.881367   
mmsyn_sm03  20         0     269.367019            6.017975       256.009099   
mmsyn_sm04  20         0     280.105800            6.229128       264.934042   
mmsyn_sm05  3          0     525.337994           23.373250       430.760837   
            5          1     297.024386           10.134587       258.887889   
            10         2     288.971560            6.328435       267.144388   
            20         3     289.940653            6.493218       272.068321   
            30         4     305.937538            6.496911       289.162569   
            40         5     302.909221            6.548644       287.116258   
            50         6     310.256550            5.311861       295.838342   
            60         7     319.054192            5.615725       304.698718   
mmsyn_sm06  20         0     301.703066            6.696372       279.274147   
mmsyn_sm07  20         0     320.104203            6.737870       287.491369   
mmsyn_sm08  20         0     344.739197            6.957466       295.322001   
mmsyn_sm09  20         0     396.763898            7.121113       310.523301   
mmsyn_sm10  3          0    3113.411413           30.326670       561.427486   
            5          1    1248.110931           13.106479       383.310370   
            10         2     650.412602            7.594858       314.011275   
            20         3     470.431632            7.388969       309.325025   
            30         4     422.792873            7.355976       316.804994   
            40         5     413.980595            7.439492       324.542484   
            50         6     408.436980            7.573976       333.083559   
            60         7     432.682803            6.275592       371.084913   
mmsyn_sm11  20         0     630.802107            7.544546       316.981338   
mmsyn_sm12  20         0     954.831503            7.795595       324.828020   
mmsyn_sm13  20         0    1382.111470            7.963332       332.207410   
mmsyn_sm14  20         0    2454.367435            8.331147       348.546360   
mmsyn_sm15  10         0    9816.267824            8.800745       383.081439   
            20         1    4850.385096            8.376897       378.167233   
            30         2    3155.184272            8.497497       355.650106   
            40         3    2480.379088            8.485936       364.384804   
            50         4    2095.922951            8.584302       372.723605   
            60     

In [17]:
# drop not needed columns
dataframe.drop(['index', 'swap'], axis=1, inplace=True)
# make table more tight
# rename columns
dataframe.rename(columns={'time_total': 'total', 'time_initial_setup': 'initial_setup', 'time_projection': 'projection', 'time_enumeration': 'enumeration', 'time_postprocessing': 'postprocessing', 'max_rss': 'max_rss [GB]'}, inplace=True)
# get RSS to MB
dataframe['max_rss [GB]'] = round(dataframe['max_rss [GB]'] / (1000 * 1000), 3)
# round time columns
cols = ['total', 'initial_setup', 'projection', 'enumeration', 'postprocessing']
dataframe[cols] = dataframe[cols].round(1)

dataframe

total  initial_setup  projection  enumeration  \
models      cores                                                     
e_coli_core 3          87.4            1.7        81.0          4.6   
            5         101.3            1.5        97.5          2.2   
            10        117.2            1.5       113.6          1.9   
            20        100.1            1.5        96.8          1.6   
            30         98.6            1.6        95.0          1.8   
            40        124.9            1.6       121.6          1.4   
            60        102.4            1.6        98.9          1.5   
mmsyn_sm00  3         381.5           17.8       354.8          3.8   
            5         279.9            8.4       265.0          1.4   
            10        246.1            5.6       234.4          1.0   
            20        253.5            5.6       241.9          0.9   
            30        262.2            5.7       250.4          0.9   
            40        268.9            5.7       257.1          0.9   
            50        285.8            4.6       275.1          0.9   
            60        284.3            4.7       273.4          0.9   
mmsyn_sm01  20        256.3            5.6       244.3          1.1   
mmsyn_sm02  20        292.3            5.9       279.9          1.4   
mmsyn_sm03  20        269.4            6.0       256.0          2.1   
mmsyn_sm04  20        280.1            6.2       264.9          3.7   
mmsyn_sm05  3         525.3           23.4       430.8         66.0   
            5         297.0           10.1       258.9         22.7   
            10        289.0            6.3       267.1         10.2   
            20        289.9            6.5       272.1          6.1   
            30        305.9            6.5       289.2          5.0   
            40        302.9            6.5       287.1          3.9   
            50        310.3            5.3       295.8          3.8   
            60        319.1            5.6       304.7          3.4   
mmsyn_sm06  20        301.7            6.7       279.3         10.3   
mmsyn_sm07  20        320.1            6.7       287.5         20.2   
mmsyn_sm08  20        344.7            7.0       295.3         36.3   
mmsyn_sm09  20        396.8            7.1       310.5         71.7   
mmsyn_sm10  3        3113.4           30.3       561.4       2487.3   
            5        1248.1           13.1       383.3        830.0   
            10        650.4            7.6       314.0        313.5   
            20        470.4            7.4       309.3        142.1   
            30        422.8            7.4       316.8         88.8   
            40        414.0            7.4       324.5         72.0   
            50        408.4            7.6       333.1         57.8   
            60        432.7            6.3       371.1         44.8   
mmsyn_sm11  20        630.8            7.5       317.0        287.9   
mmsyn_sm12  20        954.8            7.8       324.8        589.7   
mmsyn_sm13  20       1382.1            8.0       332.2        985.7   
mmsyn_sm14  20       2454.4            8.3       348.5       1988.1   
mmsyn_sm15  10       9816.3            8.8       383.1       9071.9   
            20       4850.4            8.4       378.2       4245.0   
            30       3155.2            8.5       355.7       2616.8   
            40       2480.4            8.5       364.4       1950.6   
            50       2095.9            8.6       372.7       1556.2   
            60       1849.8            7.1       383.0       1299.8   
mmsyn_sm16  20       9492.8            8.6       358.7       8685.0   
mmsyn_sm17  20      18720.3            9.2       375.2      17629.7   
mmsyn_sm18  60      10832.1            8.7       412.4       9392.8   
mmsyn_sm19  60      49735.8            9.5       429.0      47762.1   
mmsyn_sm20  60      82744.2            8.7       465.7      79924.6   
mmsyn_sm21  60     192268.0            9.0    

In [18]:
print(dataframe.to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
           &    &     total &  initial\_setup &  projection &  enumeration &  postprocessing &   ECM\_count &  max\_rss [GB] \\
models & cores &           &                &             &              &                 &             &               \\
\midrule
e\_coli\_core & 3  &      87.4 &            1.7 &        81.0 &          4.6 &             0.1 &         689 &         0.329 \\
           & 5  &     101.3 &            1.5 &        97.5 &          2.2 &             0.1 &         689 &         0.329 \\
           & 10 &     117.2 &            1.5 &       113.6 &          1.9 &             0.2 &         689 &         0.332 \\
           & 20 &     100.1 &            1.5 &        96.8 &          1.6 &             0.2 &         689 &         0.328 \\
           & 30 &      98.6 &            1.6 &        95.0 &          1.8 &             0.2 &         689 &         0.330 \\
           & 40 &     124.9 &            1.6 &       121.6 &          1.4 &

/tmp/ipykernel_3339038/597193242.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dataframe.to_latex())


#### ECMproject in single postprocessing

In [19]:
root = './'
path = root + 'times/ECMproject/single/'
mmsyn_models = ['mmsyn_sm' + str(i).zfill(2) + '.xml' for i in range(0,18)]
models = mmsyn_models

In [20]:
dataframe = pd.DataFrame()
model_names = []

for model in models:
    model_name = model.split('.')[0]
    filename = path + model_name + '_own.csv'
    table = pd.read_csv(filename)
    for index in table.index:
        model_names.append(model_name)
    dataframe = pd.concat([dataframe, table])

# set new multiindex
dataframe['models'] = model_names
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index(['models', 'cores'])
dataframe

,,index,time_total,time_initial_setup,time_projection,time_enumeration,time_postprocessing,ECM_count,max_rss,swap
models,cores,,,,,,,,,
mmsyn_sm00,20,0,438.574453,6.803320,429.837140,1.630802,0.301734,4132,250094.666667,0
mmsyn_sm01,20,0,487.076060,7.534682,477.064565,2.048329,0.426955,6486,384149.333333,0
mmsyn_sm02,20,0,491.641726,7.289215,481.258101,2.525486,0.567543,9702,347025.333333,0
mmsyn_sm03,20,0,447.115510,7.093019,435.995871,3.245632,0.779657,19394,328928.000000,0
mmsyn_sm04,20,0,470.950618,7.314412,456.543181,5.795499,1.296201,38778,376854.666667,0
mmsyn_sm05,20,0,461.631466,7.479267,440.735775,10.749226,2.665931,77546,549501.333333,0
mmsyn_sm06,20,0,477.659389,7.710090,450.417762,14.552363,4.977791,155082,683738.666667,0
mmsyn_sm07,20,0,578.934119,10.148144,532.102107,25.659133,11.023219,310154,896313.333333,0
mmsyn_sm08,20,0,550.469656,8.232102,473.093859,49.047899,20.094566,620298,860722.666667,0


In [21]:
# drop not needed columns
dataframe.drop(['index', 'swap'], axis=1, inplace=True)
# make table more tight
# rename columns
dataframe.rename(columns={'time_total': 'total', 'time_initial_setup': 'initial_setup', 'time_projection': 'projection', 'time_enumeration': 'enumeration', 'time_postprocessing': 'postprocessing', 'max_rss': 'max_rss [GB]'}, inplace=True)
# get RSS to MB
dataframe['max_rss [GB]'] = round(dataframe['max_rss [GB]'] / (1000 * 1000), 3)
# round time columns
cols = ['total', 'initial_setup', 'projection', 'enumeration', 'postprocessing']
dataframe[cols] = dataframe[cols].round(1)

dataframe

,,total,initial_setup,projection,enumeration,postprocessing,ECM_count,max_rss [GB]
models,cores,,,,,,,
mmsyn_sm00,20,438.6,6.8,429.8,1.6,0.3,4132,0.250
mmsyn_sm01,20,487.1,7.5,477.1,2.0,0.4,6486,0.384
mmsyn_sm02,20,491.6,7.3,481.3,2.5,0.6,9702,0.347
mmsyn_sm03,20,447.1,7.1,436.0,3.2,0.8,19394,0.329
mmsyn_sm04,20,471.0,7.3,456.5,5.8,1.3,38778,0.377
mmsyn_sm05,20,461.6,7.5,440.7,10.7,2.7,77546,0.550
mmsyn_sm06,20,477.7,7.7,450.4,14.6,5.0,155082,0.684
mmsyn_sm07,20,578.9,10.1,532.1,25.7,11.0,310154,0.896
mmsyn_sm08,20,550.5,8.2,473.1,49.0,20.1,620298,0.861


In [22]:
print(dataframe.to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
           &    &    total &  initial\_setup &  projection &  enumeration &  postprocessing &  ECM\_count &  max\_rss [GB] \\
models & cores &          &                &             &              &                 &            &               \\
\midrule
mmsyn\_sm00 & 20 &    438.6 &            6.8 &       429.8 &          1.6 &             0.3 &       4132 &         0.250 \\
mmsyn\_sm01 & 20 &    487.1 &            7.5 &       477.1 &          2.0 &             0.4 &       6486 &         0.384 \\
mmsyn\_sm02 & 20 &    491.6 &            7.3 &       481.3 &          2.5 &             0.6 &       9702 &         0.347 \\
mmsyn\_sm03 & 20 &    447.1 &            7.1 &       436.0 &          3.2 &             0.8 &      19394 &         0.329 \\
mmsyn\_sm04 & 20 &    471.0 &            7.3 &       456.5 &          5.8 &             1.3 &      38778 &         0.377 \\
mmsyn\_sm05 & 20 &    461.6 &            7.5 &       440.7 &         10.7 &            

/tmp/ipykernel_3339038/597193242.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dataframe.to_latex())


#### ECMproject pool

In [23]:
root = './'
path = root + 'times/ECMproject/pool/'
mmsyn_models = ['mmsyn_sm' + str(i).zfill(2) + '.xml' for i in range(0,19)]
models = mmsyn_models

In [24]:
dataframe = pd.DataFrame()
model_names = []

for model in models:
    model_name = model.split('.')[0]
    filename = path + model_name + '_own.csv'
    table = pd.read_csv(filename)
    for index in table.index:
        model_names.append(model_name)
    dataframe = pd.concat([dataframe, table])

# set new multiindex
dataframe['models'] = model_names
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index(['models', 'cores'])
dataframe

index    time_total  time_initial_setup  time_projection  \
models     cores                                                             
mmsyn_sm00 3          0    365.680363           17.803976       343.888412   
           5          1    235.457031            8.428420       225.359266   
           10         2    241.286283            5.527248       234.474747   
           20         3    258.856773            5.582874       252.041302   
           30         4    255.448840            5.684836       248.544864   
           40         5    263.687408            5.746688       256.723967   
           50         6    269.722610            4.602328       263.882688   
           60         7    278.379377            4.738159       272.341556   
mmsyn_sm01 20         0    250.388434            5.612681       243.273422   
mmsyn_sm02 20         0    257.493564            5.882829       249.739134   
mmsyn_sm03 20         0    264.355699            6.017830       255.346215   
mmsyn_sm04 20         0    274.580093            6.216895       263.405651   
mmsyn_sm05 3          0    508.212043           23.445400       413.600994   
           5          1    295.397736           10.105952       257.997706   
           10         2    288.413449            6.325338       267.080869   
           20         3    319.063974           34.596788       273.057238   
           30         4    296.604317            6.476797       279.929084   
           40         5    303.539607            6.519402       287.733277   
           50         6    310.553762            5.307263       296.107468   
           60         7    317.614913            5.611230       303.805329   
mmsyn_sm06 20         0    299.937418            6.695484       279.053799   
mmsyn_sm07 20         0    317.444076            6.763733       287.558541   
mmsyn_sm08 20         0    342.695374            7.003123       294.917099   
mmsyn_sm09 20         0    385.528955            7.168120       301.408555   
mmsyn_sm10 3          0   3114.304932           30.273623       560.881673   
           5          1   1236.682506           13.068720       372.856618   
           10         2    653.608186            7.554001       313.536558   
           20         3    508.260843            7.345698       348.039000   
           30         4    425.731421            7.397355       317.153154   
           40         5    412.037789            7.441883       325.126757   
           50         6    436.720432            7.543672       364.433605   
           60         7    403.698171            6.310515       342.814373   
mmsyn_sm11 20         0    629.875398            7.560497       316.091234   
mmsyn_sm12 20         0    952.781389            7.810356       324.503805   
mmsyn_sm13 20         0   1408.068036            7.950994       361.441672   
mmsyn_sm14 20         0   2448.293542            8.356764       339.394666   
mmsyn_sm15 10         0   9752.789398            8.798430       344.168796   
           20         1   4807.297467            8.373818       347.252873   
           30         2   3197.001051            8.495078       384.580015   
           40         3   2499.554678            8.488355       364.681445   
           50         4   2128.211055            8.581505       384.552337   
           60         5   1869.133778            7.122692       382.627323   
mmsyn_sm16 20         0   9370.932078            8.606718       358.347006   
mmsyn_sm17 20         0  18897.899024            9.107004       395.508142   
mmsyn_sm18 60         0  11157.382846            8.724495       440.695558   

                  time_enumeration  time_postprocessing  ECM_count  \
models     cores                                                     
mmsyn_sm00 3              3.734362             0.252433       4132   
           5              1.395072             0.273031       4132   
           10             0.997910             0.285166       4132   
 

In [25]:
# drop not needed columns
dataframe.drop(['index', 'swap'], axis=1, inplace=True)
# make table more tight
# rename columns
dataframe.rename(columns={'time_total': 'total', 'time_initial_setup': 'initial_setup', 'time_projection': 'projection', 'time_enumeration': 'enumeration', 'time_postprocessing': 'postprocessing', 'max_rss': 'max_rss [GB]'}, inplace=True)
# get RSS to MB
dataframe['max_rss [GB]'] = round(dataframe['max_rss [GB]'] / (1000 * 1000), 3)
# round time columns
cols = ['total', 'initial_setup', 'projection', 'enumeration', 'postprocessing']
dataframe[cols] = dataframe[cols].round(1)

dataframe

total  initial_setup  projection  enumeration  \
models     cores                                                    
mmsyn_sm00 3        365.7           17.8       343.9          3.7   
           5        235.5            8.4       225.4          1.4   
           10       241.3            5.5       234.5          1.0   
           20       258.9            5.6       252.0          0.9   
           30       255.4            5.7       248.5          0.9   
           40       263.7            5.7       256.7          0.9   
           50       269.7            4.6       263.9          0.9   
           60       278.4            4.7       272.3          0.9   
mmsyn_sm01 20       250.4            5.6       243.3          1.1   
mmsyn_sm02 20       257.5            5.9       249.7          1.4   
mmsyn_sm03 20       264.4            6.0       255.3          2.2   
mmsyn_sm04 20       274.6            6.2       263.4          3.6   
mmsyn_sm05 3        508.2           23.4       413.6         65.9   
           5        295.4           10.1       258.0         22.0   
           10       288.4            6.3       267.1          9.7   
           20       319.1           34.6       273.1          6.1   
           30       296.6            6.5       279.9          4.9   
           40       303.5            6.5       287.7          3.9   
           50       310.6            5.3       296.1          3.8   
           60       317.6            5.6       303.8          2.8   
mmsyn_sm06 20       299.9            6.7       279.1         10.7   
mmsyn_sm07 20       317.4            6.8       287.6         19.1   
mmsyn_sm08 20       342.7            7.0       294.9         35.9   
mmsyn_sm09 20       385.5            7.2       301.4         70.2   
mmsyn_sm10 3       3114.3           30.3       560.9       2491.6   
           5       1236.7           13.1       372.9        830.8   
           10       653.6            7.6       313.5        313.0   
           20       508.3            7.3       348.0        142.6   
           30       425.7            7.4       317.2         91.6   
           40       412.0            7.4       325.1         69.7   
           50       436.7            7.5       364.4         55.0   
           60       403.7            6.3       342.8         44.8   
mmsyn_sm11 20       629.9            7.6       316.1        288.6   
mmsyn_sm12 20       952.8            7.8       324.5        588.0   
mmsyn_sm13 20      1408.1            8.0       361.4        983.9   
mmsyn_sm14 20      2448.3            8.4       339.4       1989.3   
mmsyn_sm15 10      9752.8            8.8       344.2       9076.3   
           20      4807.3            8.4       347.3       4243.9   
           30      3197.0            8.5       384.6       2621.0   
           40      2499.6            8.5       364.7       1947.2   
           50      2128.2            8.6       384.6       1558.4   
           60      1869.1            7.1       382.6       1299.0   
mmsyn_sm16 20      9370.9            8.6       358.3       8673.8   
mmsyn_sm17 20     18897.9            9.1       395.5      17634.1   
mmsyn_sm18 60     11157.4            8.7       440.7       9383.2   

                  postprocessing  ECM_count  max_rss [GB]  
models     cores                                           
mmsyn_sm00 3                 0.3       4132         0.228  
           5                 0.3       4132         0.300  
           10                0.3       4132         0.300  
           20                0.3       4132         0.286  
           30                0.4       4132         0.265  
           40                0.4       4132         0.244  
           50                0.4       4132         0.264  
           60                0.4       4132         0.233  
mmsyn_sm01 20                0.4       6486         0.271  
mmsyn_sm02 20                0.5       9702         0.275  
mmsyn_sm03 20                0.8      19394         0.30

In [26]:
print(dataframe.to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
           &    &    total &  initial\_setup &  projection &  enumeration &  postprocessing &  ECM\_count &  max\_rss [GB] \\
models & cores &          &                &             &              &                 &            &               \\
\midrule
mmsyn\_sm00 & 3  &    365.7 &           17.8 &       343.9 &          3.7 &             0.3 &       4132 &         0.228 \\
           & 5  &    235.5 &            8.4 &       225.4 &          1.4 &             0.3 &       4132 &         0.300 \\
           & 10 &    241.3 &            5.5 &       234.5 &          1.0 &             0.3 &       4132 &         0.300 \\
           & 20 &    258.9 &            5.6 &       252.0 &          0.9 &             0.3 &       4132 &         0.286 \\
           & 30 &    255.4 &            5.7 &       248.5 &          0.9 &             0.4 &       4132 &         0.265 \\
           & 40 &    263.7 &            5.7 &       256.7 &          0.9 &             0.4 

/tmp/ipykernel_3339038/597193242.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dataframe.to_latex())


#### ECMproject on mfel

In [27]:
root = './'
path = root + 'times/ECMproject/mfel/'
mmsyn_models = ['mmsyn_sm' + str(i).zfill(2) + '.xml' for i in [5,10]]
models = mmsyn_models

In [28]:
dataframe = pd.DataFrame()
model_names = []

for model in models:
    model_name = model.split('.')[0]
    filename = path + model_name + '_mfel.csv'
    table = pd.read_csv(filename)
    for index in table.index:
        model_names.append(model_name)
    dataframe = pd.concat([dataframe, table])

# set new multiindex
dataframe['models'] = model_names
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index(['models', 'cores'])
dataframe

index   time_total  time_initial_setup  time_projection  \
models     cores                                                            
mmsyn_sm05 3          0   407.096781           23.507834       314.882152   
           5          1   364.367510           10.135886       329.196948   
           10         2   375.883310            6.324341       357.625407   
           20         3   474.840825            6.475509       459.385271   
           30         4   627.664185            6.459658       613.650214   
           40         5   733.949819            6.512175       720.646477   
           50         6   920.425988            5.330171       909.346060   
           60         7  1019.128416            5.551733      1008.125232   
mmsyn_sm10 3          0  2931.395770           30.266518       379.726741   
           5          1  1613.471293           15.773218       608.141057   
           10         2  1111.042559           10.131416       699.864658   
           20         3  1077.177130            8.522502       860.547722   
           30         4  1082.909154            8.401067       930.511125   
           40         5   961.303659            7.930680       873.951706   
           50         6  1007.095406            7.559691       933.237835   
           60         7  1267.881279            6.309801      1206.320251   

                  time_enumeration  time_postprocessing  ECM_count  \
models     cores                                                     
mmsyn_sm05 3             66.203647             2.501930      77546   
           5             22.534356             2.499078      77546   
           10             9.404981             2.527338      77546   
           20             6.432879             2.545916      77546   
           30             4.995693             2.557359      77546   
           40             4.171447             2.618511      77546   
           50             3.044619             2.703541      77546   
           60             2.821001             2.629198      77546   
mmsyn_sm10 3           2487.574209            33.827020    2481162   
           5            966.634230            22.921304    2481162   
           10           384.925002            16.120008    2481162   
           20           195.509429            12.596052    2481162   
           30           134.177777             9.817861    2481162   
           40            69.772537             9.647462    2481162   
           50            56.150559            10.146048    2481162   
           60            45.305275             9.944683    2481162   

                        max_rss  swap  
models     cores                       
mmsyn_sm05 3      508036.000000     0  
           5      527433.333333     0  
           10     506210.666667     0  
           20     505672.000000     0  
           30     505105.333333     0  
           40     521146.666667     0  
           50     503306.666667     0  
           60     519416.000000     0  
mmsyn_sm10 3      642621.333333     0  
           5      634512.000000     0  
           10     641662.666667     0  
           20     627126.666667     0  
           30     624118.666667     0  
           40     634557.333333     0  
           50     653442.666667     0  
           60     626104.000000     0

In [29]:
# drop not needed columns
dataframe.drop(['index', 'swap'], axis=1, inplace=True)
# make table more tight
# rename columns
dataframe.rename(columns={'time_total': 'total', 'time_initial_setup': 'initial_setup', 'time_projection': 'projection', 'time_enumeration': 'enumeration', 'time_postprocessing': 'postprocessing', 'max_rss': 'max_rss [GB]'}, inplace=True)
# get RSS to MB
dataframe['max_rss [GB]'] = round(dataframe['max_rss [GB]'] / (1000 * 1000), 3)
# round time columns
cols = ['total', 'initial_setup', 'projection', 'enumeration', 'postprocessing']
dataframe[cols] = dataframe[cols].round(1)

dataframe

total  initial_setup  projection  enumeration  \
models     cores                                                   
mmsyn_sm05 3       407.1           23.5       314.9         66.2   
           5       364.4           10.1       329.2         22.5   
           10      375.9            6.3       357.6          9.4   
           20      474.8            6.5       459.4          6.4   
           30      627.7            6.5       613.7          5.0   
           40      733.9            6.5       720.6          4.2   
           50      920.4            5.3       909.3          3.0   
           60     1019.1            5.6      1008.1          2.8   
mmsyn_sm10 3      2931.4           30.3       379.7       2487.6   
           5      1613.5           15.8       608.1        966.6   
           10     1111.0           10.1       699.9        384.9   
           20     1077.2            8.5       860.5        195.5   
           30     1082.9            8.4       930.5        134.2   
           40      961.3            7.9       874.0         69.8   
           50     1007.1            7.6       933.2         56.2   
           60     1267.9            6.3      1206.3         45.3   

                  postprocessing  ECM_count  max_rss [GB]  
models     cores                                           
mmsyn_sm05 3                 2.5      77546         0.508  
           5                 2.5      77546         0.527  
           10                2.5      77546         0.506  
           20                2.5      77546         0.506  
           30                2.6      77546         0.505  
           40                2.6      77546         0.521  
           50                2.7      77546         0.503  
           60                2.6      77546         0.519  
mmsyn_sm10 3                33.8    2481162         0.643  
           5                22.9    2481162         0.635  
           10               16.1    2481162         0.642  
           20               12.6    2481162         0.627  
           30                9.8    2481162         0.624  
           40                9.6    2481162         0.635  
           50               10.1    2481162         0.653  
           60                9.9    2481162         0.626

In [30]:
print(dataframe.to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
           &    &   total &  initial\_setup &  projection &  enumeration &  postprocessing &  ECM\_count &  max\_rss [GB] \\
models & cores &         &                &             &              &                 &            &               \\
\midrule
mmsyn\_sm05 & 3  &   407.1 &           23.5 &       314.9 &         66.2 &             2.5 &      77546 &         0.508 \\
           & 5  &   364.4 &           10.1 &       329.2 &         22.5 &             2.5 &      77546 &         0.527 \\
           & 10 &   375.9 &            6.3 &       357.6 &          9.4 &             2.5 &      77546 &         0.506 \\
           & 20 &   474.8 &            6.5 &       459.4 &          6.4 &             2.5 &      77546 &         0.506 \\
           & 30 &   627.7 &            6.5 &       613.7 &          5.0 &             2.6 &      77546 &         0.505 \\
           & 40 &   733.9 &            6.5 &       720.6 &          4.2 &             2.6 &      7

/tmp/ipykernel_3339038/597193242.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dataframe.to_latex())


#### ecmtool

In [32]:
root = './'
path = root + 'times/ecmtool/'
mmsyn_models = ['mmsyn_sm' + str(i).zfill(2) + '.xml' for i in range(0,20)]
models = ['e_coli_core.xml'] + mmsyn_models

In [33]:
dataframe = pd.DataFrame()
model_names = []

for model in models:
    model_name = model.split('.')[0]
    filename = path + model_name + '_ecmtool.csv'
    table = pd.read_csv(filename)
    for index in table.index:
        model_names.append(model_name)
    dataframe = pd.concat([dataframe, table])

# set new multiindex
dataframe['models'] = model_names
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index(['models', 'cores'])
dataframe

index     time_total  time_preprocessing  \
models      cores                                             
e_coli_core 3          0      15.544508            2.857596   
            5          1      11.778240            2.833795   
            10         2      11.245091            2.805251   
            20         3      10.568818            2.813146   
            30         4      10.777383            2.831349   
            40         5      10.594721            2.888451   
            60         6      10.579934            2.814251   
mmsyn_sm00  3          0     122.518321            3.741230   
            5          1      49.246774            3.738007   
            10         2      33.066309            3.707299   
            20         3      24.267474            3.752686   
            30         4      23.427711            3.744731   
            40         5      22.057214            3.674567   
            50         6      21.746743            3.764915   
            60         7      20.884445            3.713202   
mmsyn_sm01  20         0      26.658224            3.942038   
mmsyn_sm02  20         0      33.296191            4.007242   
mmsyn_sm03  20         0      39.618154            4.234156   
mmsyn_sm04  20         0      41.526072            4.310299   
mmsyn_sm05  3          0     463.714441            4.706722   
            5          1     162.813633            4.799160   
            10         2      88.891790            4.657139   
            20         3      57.738038            4.561257   
            30         4      49.596522            4.621302   
            40         5      48.077594            4.627138   
            50         6      45.057002            4.718734   
            60         7      44.489643            4.580946   
mmsyn_sm06  20         0      88.272588            4.822165   
mmsyn_sm07  20         0     133.636455            5.167932   
mmsyn_sm08  20         0     227.426771            4.525389   
mmsyn_sm09  20         0     386.000386            4.680428   
mmsyn_sm10  3          0    8188.396264            4.713957   
            5          1    2920.318741            4.752655   
            10         2    1291.582050            4.808309   
            20         3     736.439259            4.777594   
            30         4     564.966040            4.847406   
            40         5     491.740108            4.743434   
            50         6     459.693838            4.868129   
            60         7     430.093484            4.858643   
mmsyn_sm11  20         0    1472.199615            5.008730   
mmsyn_sm12  20         0    2926.815329            5.375788   
mmsyn_sm13  20         0    5207.284375            5.502369   
mmsyn_sm14  20         0   14739.866365            5.634465   
mmsyn_sm15  10         0   43161.284111            6.028036   
            20         1   23066.678082            6.001310   
            30         2   17377.283693            5.856179   
            40         3   14909.485083            6.006446   
            50         4   13329.193561            5.912971   
            60         5   12326.799327            5.882606   
mmsyn_sm16  20         0   73923.510940            6.034002   
mmsyn_sm17  20         0  146722.410560            6.294645   
mmsyn_sm18  60         0  141957.353909            6.563696   
mmsyn_sm19  60         0  851414.161233            6.908262   

                   time_first_vertex_enumeration  \
models      cores                                  
e_coli_core 3                           1.600098   
            5                           1.213874   
            10                          1.185267   
            20                          1.219440   
            30                          1.318640   
            40                          1.282195   
            60                          1.389834   
mmsyn_sm00  3                          99.876917   
            5                    

In [34]:
# drop not needed columns
dataframe.drop(['index', 'swap'], axis=1, inplace=True)
# make table more tight
# rename columns
dataframe.rename(columns={'time_total': 'total', 'time_preprocessing': 'preprocessing', 'time_first_vertex_enumeration': 'first_vertex_enumeration', 'time_intermediate_processing': 'intermediate_processing', 'time_second_vertex_enumeration': 'second_vertex_enumeration', 'time_postprocessing': 'postprocessing', 'max_rss': 'max_rss [GB]'}, inplace=True)
# get RSS to MB
dataframe['max_rss [GB]'] = round(dataframe['max_rss [GB]'] / (1000 * 1000), 3)
# round time columns
cols = ['total', 'preprocessing', 'first_vertex_enumeration', 'intermediate_processing', 'second_vertex_enumeration', 'postprocessing']
dataframe[cols] = dataframe[cols].round(1)

dataframe

total  preprocessing  first_vertex_enumeration  \
models      cores                                                      
e_coli_core 3          15.5            2.9                       1.6   
            5          11.8            2.8                       1.2   
            10         11.2            2.8                       1.2   
            20         10.6            2.8                       1.2   
            30         10.8            2.8                       1.3   
            40         10.6            2.9                       1.3   
            60         10.6            2.8                       1.4   
mmsyn_sm00  3         122.5            3.7                      99.9   
            5          49.2            3.7                      31.8   
            10         33.1            3.7                      17.1   
            20         24.3            3.8                       9.5   
            30         23.4            3.7                       9.0   
            40         22.1            3.7                       7.9   
            50         21.7            3.8                       7.4   
            60         20.9            3.7                       6.8   
mmsyn_sm01  20         26.7            3.9                       9.0   
mmsyn_sm02  20         33.3            4.0                      16.5   
mmsyn_sm03  20         39.6            4.2                      17.3   
mmsyn_sm04  20         41.5            4.3                      15.3   
mmsyn_sm05  3         463.7            4.7                     202.5   
            5         162.8            4.8                      62.5   
            10         88.9            4.7                      33.6   
            20         57.7            4.6                      17.6   
            30         49.6            4.6                      15.1   
            40         48.1            4.6                      14.3   
            50         45.1            4.7                      11.3   
            60         44.5            4.6                      11.2   
mmsyn_sm06  20         88.3            4.8                      21.9   
mmsyn_sm07  20        133.6            5.2                      19.9   
mmsyn_sm08  20        227.4            4.5                      20.4   
mmsyn_sm09  20        386.0            4.7                      26.5   
mmsyn_sm10  3        8188.4            4.7                     255.5   
            5        2920.3            4.8                      78.1   
            10       1291.6            4.8                      41.1   
            20        736.4            4.8                      26.1   
            30        565.0            4.8                      18.7   
            40        491.7            4.7                      17.3   
            50        459.7            4.9                      14.7   
            60        430.1            4.9                      12.4   
mmsyn_sm11  20       1472.2            5.0                      28.2   
mmsyn_sm12  20       2926.8            5.4                      28.2   
mmsyn_sm13  20       5207.3            5.5                      29.7   
mmsyn_sm14  20      14739.9            5.6                      29.9   
mmsyn_sm15  10      43161.3            6.0                      61.2   
            20      23066.7            6.0                      29.3   
            30      17377.3            5.9                      27.4   
            40      14909.5            6.0                      23.9   
            50      13329.2            5.9                      19.2   
            60      12326.8            5.9                      18.0   
mmsyn_sm16  20      73923.5            6.0                      38.2   
mmsyn_sm17  20     146722.4            6.3                      34.8   
mmsyn_sm18  60     141957.4            6.6                      19.6   
mmsyn_sm19  60     851414.2            6.9                      20.5   

                   intermediate_processing  second_vertex_enu

In [35]:
print(dataframe.to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
           &    &     total &  preprocessing &  first\_vertex\_enumeration &  intermediate\_processing &  second\_vertex\_enumeration &  postprocessing &  max\_rss [GB] \\
models & cores &           &                &                           &                          &                            &                 &               \\
\midrule
e\_coli\_core & 3  &      15.5 &            2.9 &                       1.6 &                      4.8 &                        6.2 &             0.0 &         0.370 \\
           & 5  &      11.8 &            2.8 &                       1.2 &                      4.9 &                        2.8 &             0.0 &         0.369 \\
           & 10 &      11.2 &            2.8 &                       1.2 &                      5.0 &                        2.2 &             0.0 &         0.366 \\
           & 20 &      10.6 &            2.8 &                       1.2 &                      4.9 &                

/tmp/ipykernel_3339038/597193242.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dataframe.to_latex())


### Create data table for zipped vs unzipped comprison

All data written in the dictionary below are taken from result files created by ECMproject. These result files got deleted afterwards to save space. They can easiely be reproduced with ECMproject. <br>Number of repeats: 1

In [77]:
gzip_dict = {
    'e_coli_core':
            {
            '20':
                {'filesize_zipped [GB]': '15.9e-06', 'filesize_unzipped [GB]': '96.8e-06', 'postprocessing [s]': 31.215341329574585}
            },
    'mmsyn_sm05':
            {
            '20':
                {'filesize_zipped [GB]': '1.2e-03', 'filesize_unzipped [GB]': '34.7e-03', 'postprocessing [s]': 35.3887734413147}
            },
    'mmsyn_sm10':
            {
            '20':
                {'filesize_zipped [GB]': '37.1e-03', 'filesize_unzipped [GB]': '1.2', 'postprocessing [s]': 163.0174970626831}
            }, 
    'mmsyn_sm15':
            {'20':
                {'filesize_zipped [GB]': '1.1', 'filesize_unzipped [GB]': '35.7', 'postprocessing [s]': 3919.637202978134}
            }
    }

In [78]:
gzip_dict

{'e_coli_core': {'20': {'filesize_zipped [GB]': '15.9e-06',
   'filesize_unzipped [GB]': '96.8e-06',
   'postprocessing [s]': 31.215341329574585}},
 'mmsyn_sm05': {'20': {'filesize_zipped [GB]': '1.2e-03',
   'filesize_unzipped [GB]': '34.7e-03',
   'postprocessing [s]': 35.3887734413147}},
 'mmsyn_sm10': {'20': {'filesize_zipped [GB]': '37.1e-03',
   'filesize_unzipped [GB]': '1.2',
   'postprocessing [s]': 163.0174970626831}},
 'mmsyn_sm15': {'20': {'filesize_zipped [GB]': '1.1',
   'filesize_unzipped [GB]': '35.7',
   'postprocessing [s]': 3919.637202978134}}}

In [79]:
gzip_table = pd.DataFrame.from_dict({(i,j): gzip_dict[i][j]
                           for i in gzip_dict.keys() 
                           for j in gzip_dict[i].keys()},
                       orient='index')

In [80]:
# change number of decimals for specific columns and change specific columns to scientific notation
gzip_table['postprocessing [s]'] = gzip_table['postprocessing [s]'].apply('{:.1f}'.format)
gzip_table

,,filesize_zipped [GB],filesize_unzipped [GB],postprocessing [s]
e_coli_core,20,15.9e-06,96.8e-06,31.2
mmsyn_sm05,20,1.2e-03,34.7e-03,35.4
mmsyn_sm10,20,37.1e-03,1.2,163.0
mmsyn_sm15,20,1.1,35.7,3919.6


In [81]:
print(gzip_table.to_latex())

\begin{tabular}{lllll}
\toprule
           &    & filesize\_zipped [GB] & filesize\_unzipped [GB] & postprocessing [s] \\
\midrule
e\_coli\_core & 20 &             15.9e-06 &               96.8e-06 &               31.2 \\
mmsyn\_sm05 & 20 &              1.2e-03 &               34.7e-03 &               35.4 \\
mmsyn\_sm10 & 20 &             37.1e-03 &                    1.2 &              163.0 \\
mmsyn\_sm15 & 20 &                  1.1 &                   35.7 &             3919.6 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_999017/2881627683.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(gzip_table.to_latex())
